# Rotina da Gerencial de acessos

Essa rotina analisa os usuários que necessitam ser atualizados e/ou criados e gera um arquivo final para subir na Skore de forma manual.

Bases de apoio:

    - Base de Alunos 
    - Base de username e userid Skore
    - Base manual da rotina da gerencial com times, cadastros, suportes e cargos
    

In [1551]:
import pandas as pd
import numpy as np
import datetime as dt
import datetime

In [1552]:
#particoes = input(str('Gostaria de particionar o envio ? '))
particoes = 's'

In [1553]:
DT_TODAY = dt.datetime.today()
DT_TODAY_MONTH_ANT = dt.datetime.today() - dt.timedelta(days = 20)
TODAY = DT_TODAY.strftime('%Y%m%d')
DT_YESTERDAY = DT_TODAY - dt.timedelta(days = 1)
YESTERDAY = DT_YESTERDAY.strftime('%Y%m%d')
MONTH = DT_TODAY.strftime('%Y%m')
MONTH_INT = int(DT_TODAY.strftime('%m'))
MONTH_STR = DT_TODAY.strftime('%m')
MONTH_COMP = DT_TODAY.strftime('%m.%Y')
MONTH_COMP_2 = DT_TODAY_MONTH_ANT.strftime('%m.%Y')
DT_TODAY_BVBV60 = dt.datetime.today() + dt.timedelta(days = 60)


In [1554]:
MATRICULA='2103510956'

## Importando as bases

In [1555]:
df_roteirizacao = pd.read_csv('C:/Users/{}/Via Varejo S.A/MARILZA ALMEIDA TONIATO - fixed_data/roteirizacao.csv'.format(MATRICULA), sep = ';')
df_base_fun = pd.read_excel('./input/{}/Alunos_{}.xlsx'.format(MONTH,TODAY))
df_user_id = pd.read_csv('./input/{}/username_userid.csv'.format(MONTH), sep = ';',encoding = 'latin-1')
df_meta_escala = pd.read_csv('C:/Users/{}/Via Varejo S.A/MARILZA ALMEIDA TONIATO - fixed_data/meta_escala_diaria.csv'.format(MATRICULA),encoding = 'latin-1')
df_suporte = pd.read_excel('./Gerencial_Manual.xlsx', sheet_name = 'Suporte')
df_cargos = pd.read_excel('./Gerencial_Manual.xlsx', sheet_name = 'Cargos')
df_times_personalizados = pd.read_excel('./Gerencial_Manual.xlsx', sheet_name = 'Times_personalizados')
df_cadastro_manual = pd.read_excel('./Gerencial_Manual.xlsx', sheet_name = 'Cadastro_manual')

In [1556]:
df_base_fun.to_excel('C:/Users/{}/Via Varejo S.A/MARILZA ALMEIDA TONIATO - fixed_data/alunos.xlsx'.format(MATRICULA), index = False)

In [1557]:
#Teste Marilza

#df_roteirizacao = pd.read_csv('C:/Users/2102709830/OneDrive - Via Varejo S.A/Área de Trabalho/Evva2.0/evva2.0/fixed_data/roteirizacao.csv', sep = ';')
#df_base_fun = pd.read_excel('./input/{}/Alunos_{}.xlsx'.format(MONTH,TODAY))
#df_user_id = pd.read_csv('./input/{}/username_userid.csv'.format(MONTH), sep = ';',encoding = 'latin-1')
#df_meta_escala = pd.read_csv('C:/Users/2102709830/OneDrive - Via Varejo S.A/Área de Trabalho/Evva2.0/evva2.0/fixed_data/meta_escala_diaria.csv',encoding = 'latin-1')
#df_suporte = pd.read_excel('./Gerencial_Manual.xlsx', sheet_name = 'Suporte')
#df_cargos = pd.read_excel('./Gerencial_Manual.xlsx', sheet_name = 'Cargos')
#df_times_personalizados = pd.read_excel('./Gerencial_Manual.xlsx', sheet_name = 'Times_personalizados')
#df_cadastro_manual = pd.read_excel('./Gerencial_Manual.xlsx', sheet_name = 'Cadastro_manual')

## Tratamento base de Funcionarios

In [1558]:
#Filtrando somente terceiros 77 e colaboradores via [21]
lista_empresa = [21,77]
df_funcionarios = df_base_fun.loc[df_base_fun['Código Empresa'].isin(lista_empresa)].copy()
#Limpando a bandeira
df_funcionarios['Bandeira'] = df_funcionarios['Bandeira'].apply(lambda x: x.split("  ")[0])

In [1559]:
#substituir os nomes das bandeiras para o padrão
depara_bandeira = {
    'CASASBAHIA':'CB',
    'PONTO':'PF',
    'PONTOFRIO':'PF',
    'CORPORATIVO':'CORP',
    'CASAS BAHIA': 'CB'
}
df_funcionarios = df_funcionarios.replace({"Bandeira": depara_bandeira})

In [1560]:
#Transformando as colunas em string para melhor trabalhabilidade
colunas = ['Código Empresa', 'Código Estabelecimento', 'Matrícula']

for coluna in colunas:
    df_funcionarios[coluna] = df_funcionarios[coluna].astype(str)
    df_funcionarios[coluna] = df_funcionarios[coluna].apply(lambda x: x.replace('.0', ''))

In [1561]:
#Ajuntando o nome
df_funcionarios['Nome do aluno'] = df_funcionarios['Nome do aluno'] + ' ' + df_funcionarios['Sobrenome do aluno'] 

### Categorias principais

In [1562]:
## Trazendo as colunas da meta escala para usar
df_vendedores = df_meta_escala[['CD_FUN','SUB_CAT_PRINCIPAL','CD_FIL']].copy()
df_vendedores['SUB_CAT_PRINCIPAL'] = df_vendedores['SUB_CAT_PRINCIPAL'].apply(lambda x: x.title())

In [1563]:
#Padronizando as colunas para o merge
df_vendedores = df_vendedores.rename(columns = {'CD_FUN':'matricula'})
df_vendedores['matricula'] = df_vendedores['matricula'].astype(str)
df_funcionarios = df_funcionarios.rename(columns = {'Matrícula':'matricula'})
df_funcionarios['matricula'] = df_funcionarios['matricula'].astype(str)

# Merge para trazer a filial dos vendedores e nome
df_base_fun = df_funcionarios.merge(df_vendedores, on = 'matricula', how = 'left')

### Filiais

#### Vendedores

In [1564]:
# Trazendo as informações da coluna de filial por funcionario para uma só coluna
#Priorizando a filial do meta escala, caso o colaborador não tenha filial cadastrada, será considerada a filial de cadastro
df_base_fun['Código Estabelecimento'] = np.where(df_base_fun['CD_FIL'].isnull(), df_base_fun['Código Estabelecimento'], df_base_fun['CD_FIL'])

#### Gerente

In [1565]:
## Trazendo as colunas para usar
df_gerente = df_roteirizacao[['CD_FIL','CD_REG','CD_DIR','CD_FUN_GER','NOME_GERENTE_LOJA']]

In [1566]:
#Alterando nome da coluna de matricula para "matricula"
df_gerente = df_gerente.rename(columns = {'CD_FUN_GER':'matricula'})
df_gerente['matricula'] = df_gerente['matricula'].astype(str)

In [1567]:
##Merge para trazer a filial dos vendedores e nome
df_base_fun = df_base_fun.merge(df_gerente, on = 'matricula', how = 'left')

In [1568]:
## Trazendo as informações da coluna de filial por funcionario para uma só coluna
df_base_fun['Código Estabelecimento'] = np.where(df_base_fun['CD_FIL_y'].isnull(), df_base_fun['Código Estabelecimento'], df_base_fun['CD_FIL_y'])

In [1569]:
## Trazer as colunas para trabalhar
df_base_fun = df_base_fun[['Código Empresa','Login do aluno','matricula','Nome do aluno','Descrição Cargo','Nascimento','CPF','Data de admissão','Situação do Colaborador','Código Estabelecimento','Bandeira','SUB_CAT_PRINCIPAL']]

#### CAL

In [1570]:
## Trazendo as colunas para usar
df_cal = df_roteirizacao[['CD_FIL','CD_REG','CD_DIR','CD_FUN_CAL','NOME_CAL']]

In [1571]:
#Alterando nome da coluna de matricula para "matricula"
df_cal = df_cal.rename(columns = {'CD_FUN_CAL':'matricula'})
df_cal['matricula'] = df_cal['matricula'].astype(str)

In [1572]:
##Merge para trazer a filial das CALs 
df_base_fun = df_base_fun.merge(df_cal, on = 'matricula', how = 'left')

In [1573]:
## Trazendo as informações da coluna de filial por funcionario para uma só coluna
df_base_fun['Código Estabelecimento'] = np.where(df_base_fun['CD_FIL'].isnull(), df_base_fun['Código Estabelecimento'], df_base_fun['CD_FIL'])

In [1574]:
## Trazer as colunas para trabalhar
df_base_fun = df_base_fun[['Código Empresa','Login do aluno','matricula','Nome do aluno','Descrição Cargo','Nascimento','CPF','Data de admissão','Situação do Colaborador','Código Estabelecimento','Bandeira','SUB_CAT_PRINCIPAL']]

### Transformando dados

In [1575]:
## Renomemando colunas
df_Oficial_fun = df_base_fun.rename(columns = {'Código Empresa':'Empresa',
                                               'Login do aluno':'username',
                                               'matricula':'Matrícula',
                                               'Nome do aluno':'Nome',
                                               'Descrição Cargo':'Cargo',
                                               'Nascimento':'Data_Nascimento',
                                               'CPF':'CPF',
                                               'Data de admissão':'Data_de_admissão',
                                               'Situação do Colaborador':'Status',
                                               'Código Estabelecimento':'Filial',
                                               'Bandeira':'Bandeira',
                                               'SUB_CAT_PRINCIPAL':'Categoria'})

In [1576]:
##Ajustando CPF
df_Oficial_fun['CPF'] = df_Oficial_fun['CPF'].astype('str')
df_Oficial_fun['CPF'] = df_Oficial_fun['CPF'].apply(lambda x: x.replace('.0', ''))

In [1577]:
df_Oficial_fun['username'] = df_Oficial_fun['username'].astype('string')

In [1578]:
##Merge para trazer user_id
df_Oficial_fun = df_Oficial_fun.merge(df_user_id, on = 'username', how='left')

In [1579]:
## Trazendo as colunas para incluir na "Oficial_fun" a diretoria e regional de cada filial
df_roteirizacao = df_roteirizacao[['CD_FIL','CD_REG','CD_DIR']]
##Alterando nomenclatura da coluna CD_Fil para trazer o merge
df_roteirizacao = df_roteirizacao.rename(columns = {'CD_FIL':'Filial',
                                                    'CD_REG':'Regional',
                                                    'CD_DIR':'Diretoria'})

In [1580]:
df_Oficial_fun['Filial'] = df_Oficial_fun['Filial'].astype('str')
df_Oficial_fun['Filial'] = df_Oficial_fun['Filial'].apply(lambda x: x.replace('.0', ''))
df_roteirizacao['Filial'] = df_roteirizacao['Filial'].astype('str')
df_roteirizacao['Filial'] = df_roteirizacao['Filial'].apply(lambda x: x.replace('.0', ''))

In [1581]:
##Meger para trazer os dados regionais
df_Oficial_fun = df_Oficial_fun.merge(df_roteirizacao, on = 'Filial', how = 'left')

In [1582]:
#Substituindo NAN para 0
df_Oficial_fun = df_Oficial_fun.replace(np.nan,'-')

In [1583]:
df_Oficial_fun = df_Oficial_fun[['Empresa','username','id','Matrícula','Nome','Cargo','Filial','Regional','Diretoria','Data_Nascimento','CPF','Data_de_admissão','Status','Bandeira','Categoria']]

In [1584]:
df_Oficial_fun = df_Oficial_fun.rename(columns = {'id':'User_id'})

In [1585]:
##Ajustando (O.0)
df_Oficial_fun['User_id'] = df_Oficial_fun['User_id'].astype('str')
df_Oficial_fun['User_id'] = df_Oficial_fun['User_id'].apply(lambda x: x.replace('.0', ''))
df_Oficial_fun['Diretoria'] = df_Oficial_fun['Diretoria'].astype('str')
df_Oficial_fun['Diretoria'] = df_Oficial_fun['Diretoria'].apply(lambda x: x.replace('.0', ''))
df_Oficial_fun['Regional'] = df_Oficial_fun['Regional'].astype('str')
df_Oficial_fun['Regional'] = df_Oficial_fun['Regional'].apply(lambda x: x.replace('.0', ''))

In [1586]:
df_Oficial_fun.head(2)

,Empresa,username,User_id,Matrícula,Nome,Cargo,Filial,Regional,Diretoria,Data_Nascimento,CPF,Data_de_admissão,Status,Bandeira,Categoria
0,21,2104482816,953877,4482816,ALINE ZUVERANA GOMES,ASSESSOR DE VENDAS,2192,414,4,10.08.1987,7716689614,16.03.2020,ATIVO,CB,Tecnologia
1,21,2160010026,2260127,60010026,OSMAR ZUMPANO NETO,BOMBEIRO,650,-,-,24.10.1990,39241369809,02.05.2022,ATIVO,CORP,-


In [1587]:
df_Oficial_fun['Data_de_admissão'] = df_Oficial_fun['Data_de_admissão'].str.replace('.','/')

C:\Users\2103510956\AppData\Local\Temp\ipykernel_5160\3086712449.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_Oficial_fun['Data_de_admissão'] = df_Oficial_fun['Data_de_admissão'].str.replace('.','/')


In [1588]:
df_Oficial_fun['Data_de_admissão']= pd.to_datetime(df_Oficial_fun['Data_de_admissão'].str.strip(), format='%d/%m/%Y')

## Criando Gerencial de Acessos

In [1589]:
df_Gerencial = df_Oficial_fun.copy()

In [1590]:
df_Gerencial['Cargo'] = df_Gerencial['Cargo'].apply(lambda x: x.split("  ")[0])

In [1591]:
## Separando apenas corporativo e operacoes de loja
df_Gerencial_operacoes = df_Gerencial[df_Gerencial['Cargo'].isin(df_cargos['Cargo'])]
corp = ['1580', '650']
df_Gerencial_corp = df_Gerencial[df_Gerencial['Filial'].isin(corp)]

df_Gerencial = pd.concat([df_Gerencial_operacoes, df_Gerencial_corp], ignore_index=True, axis = 0)

In [1592]:
#Retirando o np.nan

df_Gerencial_final = df_Gerencial.replace(np.nan, '')

In [1593]:
#Trazendo a base do suporte para ações na base
df_suporte = df_suporte.rename(columns = {'Matrícula': 'username'})
df_suporte = df_suporte.replace("Atualizar", "OVERWRITE")
df_suporte = df_suporte.replace("Cadastrar", "CREATE")
df_suporte = df_suporte[df_suporte['Status'].isnull()]
df_suporte = df_suporte.drop(columns='Status')
df_suporte['username'] = df_suporte['username'].astype(str)
df_suporte['username'] = df_suporte['username'].apply(lambda x: x.replace('.0', ''))
df_Gerencial_final = df_Gerencial_final.merge(df_suporte, on = 'username', how = 'left')
df_Gerencial_final['action'] = 'NO'
df_Gerencial_final['action'] =  np.where(df_Gerencial_final['Ação'].isnull(), df_Gerencial_final['action'], df_Gerencial_final['Ação'])

In [1594]:
df_Gerencial_final.shape

(33690, 17)

In [1595]:
#Retirando os espaços vazios do Status
df_Gerencial_final['Status'] = df_Gerencial_final['Status'].apply(lambda x: x.split("  ")[0])

In [1596]:
#df_Gerencial_final.to_excel('./df_Gerencial_final.xlsx', index = False)

In [1597]:
## Atualizar o action para os users CREATE
df_Gerencial_final['action'] = df_Gerencial_final['User_id'].apply(lambda x: 'CREATE' if x == '-' else 'NO')
df_Gerencial_final['action'] = np.where((df_Gerencial_final['action'] == "NO") & (df_Gerencial_final['Ação'] == "OVERWRITE"), 'OVERWRITE',df_Gerencial_final['action'])

In [1598]:
#Incluindo role
df_Gerencial_final['role'] = 'student'

In [1599]:
df_Gerencial_final = df_Gerencial_final.drop(columns = "Ação")

In [1600]:
df_de_para = df_cargos.copy()

In [1601]:
##Mostrando como ficou o de_para
df_de_para.head()

,Cargo,TIME_GALERIA
0,ASSIST DIRETORIA,Staff Diretoria
1,APRENDIZ ADMINISTRATIVO,Jovem Aprendiz
2,APRENDIZ ASSIST ADMINISTRATIVO,Jovem Aprendiz
3,APRENDIZ AUX ADMINISTRATIVO,Jovem Aprendiz
4,APRENDIZ VEND COM VAREJISTA I,Jovem Aprendiz


In [1602]:
##Merge para trazer os times
df_Gerencial_final = df_Gerencial_final.merge(df_de_para, on = 'Cargo', how = 'left')

In [1603]:
df_Gerencial_final.head(5)

,Empresa,username,User_id,Matrícula,Nome,Cargo,Filial,Regional,Diretoria,Data_Nascimento,CPF,Data_de_admissão,Status,Bandeira,Categoria,action,role,TIME_GALERIA
0,21,2104482816,953877,4482816,ALINE ZUVERANA GOMES,ASSESSOR DE VENDAS,2192,414,4,10.08.1987,7716689614,2020-03-16,ATIVO,CB,Tecnologia,NO,student,#Vendedor
1,21,2100240095,285815,240095,FRANCISCA ZULENE DE MELO,VENDEDOR INTERNO,1110,221,2,30.06.1978,8084588761,2003-06-18,ATIVO,CB,Linha Leve,NO,student,#Vendedor
2,21,2100240095,285815,240095,FRANCISCA ZULENE DE MELO,VENDEDOR INTERNO,1166,221,2,30.06.1978,8084588761,2003-06-18,ATIVO,CB,Linha Branca,NO,student,#Vendedor
3,21,2101839616,402426,1839616,MARIA ZULEIDE DA COSTA XAVIER,CAIXA,1707,713,7,27.04.1967,35711167391,2011-11-01,ATIVO,CB,-,NO,student,Caixa
4,21,2104908007,1248777,4908007,REGINA ZORRILLA,CONS ADM LOJA,2119,421,4,14.12.1985,2276304162,2021-02-08,ATIVO,CB,-,NO,student,CAL


In [1604]:
##Deixando tabela em padão
#incluindo argumentos obrigatórios
df_Gerencial_final['SISTEMA_PROVE'] = 'Sistema Prove'
df_Gerencial_final['GALERIA'] = 'Nova Galeria'
df_Gerencial_final['PONTO_ONLINE'] = 'Ponto Online'
df_Gerencial_final = df_Gerencial_final.rename(columns = {
    'TIME GALERIA' : 'TIME_GALERIA',
    'LIDERES DO FUTURO' : 'LIDERES_DO_FUTURO'
})

In [1605]:
df_Gerencial_final = df_Gerencial_final.rename(columns = {'Categoria':'CATEGORIA',
                                                          'Bandeira':'BANDEIRA'})

In [1606]:
df_Gerencial_final['BANDEIRA'] = np.where(df_Gerencial_final['BANDEIRA'] == 'CB', '#CasasBahia', df_Gerencial_final['BANDEIRA'])
df_Gerencial_final['BANDEIRA'] = np.where(df_Gerencial_final['BANDEIRA'] == 'PF', '#Ponto', df_Gerencial_final['BANDEIRA'])
df_Gerencial_final['BANDEIRA'] = np.where(df_Gerencial_final['BANDEIRA'] == 'Corp', 'Corporativo', df_Gerencial_final['BANDEIRA'])

In [1607]:
#df_Gerencial_final['Data_de_admissão'] = df_Gerencial_final['Data_de_admissão'].apply(lambda x: x[3:])

#### Incluindo BVBV

In [1608]:
df_Gerencial_final.BANDEIRA.value_counts()

#CasasBahia    27528
#Ponto          3835
CORP            2327
Name: BANDEIRA, dtype: int64

In [1609]:
df_Gerencial_final.BANDEIRA.value_counts()

#CasasBahia    27528
#Ponto          3835
CORP            2327
Name: BANDEIRA, dtype: int64

In [1610]:
#Trazendo o time de usuários novos por bandeira
#BVBV Vendas PT
df_Gerencial_final['BANDEIRA'] = (
        np.where((df_Gerencial_final['BANDEIRA'] == '#Ponto') &
                 (df_Gerencial_final['Data_de_admissão'] >= '2023.06.01') &
                 (DT_TODAY <= DT_TODAY_BVBV60)&
                 ((df_Gerencial_final['Cargo']=='ASSESSOR DE VENDAS') |
                 (df_Gerencial_final['Cargo']=='VENDEDOR INTERNO')), 
                 df_Gerencial_final['BANDEIRA'] + '//BVBV Vendas PT', 
                 df_Gerencial_final['BANDEIRA'])
)

#BVBV Atendimento PT
df_Gerencial_final['BANDEIRA'] = (
        np.where((df_Gerencial_final['BANDEIRA'] == '#Ponto') &
                 (df_Gerencial_final['Data_de_admissão'] >= '2023.06.01') &
                 (DT_TODAY <= DT_TODAY_BVBV60)&
                 ((df_Gerencial_final['Cargo']=='ASSESSOR DE ATENDIMENTO') |
                 (df_Gerencial_final['Cargo']=='AUX ESTOQUE')|
                 (df_Gerencial_final['Cargo']=='CAIXA')), 
                 df_Gerencial_final['BANDEIRA'] + '//BVBV Atendimento PT', 
                 df_Gerencial_final['BANDEIRA'])
)

#BVBV Vendas CB
df_Gerencial_final['BANDEIRA'] = (
        np.where((df_Gerencial_final['BANDEIRA'] == '#CasasBahia') &
                 (df_Gerencial_final['Data_de_admissão'] >= '2023.06.01') &
                 (DT_TODAY <= DT_TODAY_BVBV60)&
                 ((df_Gerencial_final['Cargo']=='ASSESSOR DE VENDAS') |
                 (df_Gerencial_final['Cargo']=='VENDEDOR INTERNO')), 
                 df_Gerencial_final['BANDEIRA'] + '//BVBV Vendas CB', 
                 df_Gerencial_final['BANDEIRA'])
)

#BVBV Atendimento CB
df_Gerencial_final['BANDEIRA'] = (
        np.where((df_Gerencial_final['BANDEIRA'] == '#CasasBahia') &
                 (df_Gerencial_final['Data_de_admissão'] >= '2023.06.01') &
                 (DT_TODAY <= DT_TODAY_BVBV60)&
                 ((df_Gerencial_final['Cargo']=='ASSESSOR DE ATENDIMENTO') |
                 (df_Gerencial_final['Cargo']=='AUX ESTOQUE')|
                 (df_Gerencial_final['Cargo']=='CAIXA')), 
                 df_Gerencial_final['BANDEIRA'] + '//BVBV Atendimento CB', 
                 df_Gerencial_final['BANDEIRA'])
)

In [1611]:
# #Trazendo o time de usuários novos por bandeira
# df_Gerencial_final['BANDEIRA'] = np.where((df_Gerencial_final['BANDEIRA'] == 'PF') & (df_Gerencial_final['Data_de_admissão'] == MONTH_COMP) , df_Gerencial_final['BANDEIRA'] + '//Novatos PONTO', df_Gerencial_final['BANDEIRA'])
# df_Gerencial_final['BANDEIRA'] = np.where((df_Gerencial_final['BANDEIRA'] == 'PF') & (df_Gerencial_final['Data_de_admissão'] == MONTH_COMP_2) , df_Gerencial_final['BANDEIRA'] + '//Novatos PONTO', df_Gerencial_final['BANDEIRA'])
# df_Gerencial_final['BANDEIRA'] = np.where((df_Gerencial_final['BANDEIRA'] == 'CB') & (df_Gerencial_final['Data_de_admissão'] == MONTH_COMP) , df_Gerencial_final['BANDEIRA'] + '//Novatos CB', df_Gerencial_final['BANDEIRA'])
# df_Gerencial_final['BANDEIRA'] = np.where((df_Gerencial_final['BANDEIRA'] == 'CB') & (df_Gerencial_final['Data_de_admissão'] == MONTH_COMP_2) , df_Gerencial_final['BANDEIRA'] + '//Novatos CB', df_Gerencial_final['BANDEIRA'])

In [1612]:
df_Gerencial_final = df_Gerencial_final[['action', 'role', 'username', 'Matrícula', 'User_id', 'Nome', 'Cargo','Data_Nascimento','CPF', 'TIME_GALERIA', 'SISTEMA_PROVE','CATEGORIA', 'GALERIA', 'PONTO_ONLINE', 'BANDEIRA','Diretoria','Regional','Filial']]

In [1613]:
df_Gerencial_final = df_Gerencial_final.rename(columns = {'username': "Username"})

### Incluindo times personalizados

In [1614]:
#Verificando a validade do time

#Separando os grupos sem validade
df_times_personalizados_final = df_times_personalizados[df_times_personalizados["Validade"] == "Sem validade"]

#Verificando os times com validade
df_times_personalizados_validade = df_times_personalizados[df_times_personalizados["Validade"] != "Sem validade"]

if df_times_personalizados_validade.shape[0] > 0:
    df_times_personalizados_validade['Validade'] = df_times_personalizados_validade['Validade'].astype(int)
    df_times_personalizados_validade['Validade'] = df_times_personalizados_validade[df_times_personalizados_validade['Validade'] >= int(TODAY)]
    df_times_personalizados_final = pd.concat([df_times_personalizados_final, df_times_personalizados_validade], axis = 0, ignore_index=True)
    
    
df_times_personalizados_final = df_times_personalizados_final[['Nome do time', 'Tipo', 'Métrica']]

In [1615]:
#Gerando as listas de comparação
tipos = df_times_personalizados_final["Tipo"].unique()
times_personalizados = df_times_personalizados_final["Nome do time"].unique()

In [1616]:
#Para cada tipo de métrica será incluso na base principal
for tipo in tipos:
    df = df_times_personalizados[df_times_personalizados['Tipo'] == tipo]
    df = df.rename(columns = {"Métrica": tipo})
    df = pd.pivot_table(df, index = tipo, columns='Nome do time', aggfunc='count', values = 'Tipo').reset_index()
    df = df.replace(np.nan, '')
    
    #Deixando todos no mesmo tipo
    df[tipo] = df[tipo].apply(lambda x: str(x).replace('.0',''))
    df_Gerencial_final[tipo] = df_Gerencial_final[tipo].apply(lambda x: str(x).replace('.0',''))
    
    #Trazendo os times para a base principal
    df_Gerencial_final = df_Gerencial_final.merge(df, on = tipo, how = 'left')

In [1617]:
df_Gerencial_final = df_Gerencial_final.replace(np.nan, '')

In [1618]:
for time in times_personalizados:
    df_Gerencial_final[time] = np.where(df_Gerencial_final[time] != '', time, '')

### Gerando a base da Skore

In [1619]:
df_usuarios = df_Gerencial_final.copy()

In [1620]:
df_usuarios = df_usuarios.rename(columns = {'Username':'username',
                                            'User_id':'user_id',
                                            'Matrícula':'matricula',
                                            'Nome':'name',
                                            'Cargo':'cargo',
                                            'Filial':'filial',
                                            'Regional':'regional',
                                            'Status':'status',
                                            'Diretoria':'diretoria',
                                            'Data_Nascimento':'nascimento',
                                            'CPF':'cpf'})

In [1621]:
df_usuarios.head(2)

,action,role,username,matricula,user_id,name,cargo,nascimento,cpf,TIME_GALERIA,...,Cipa,EstoqueDropoff,EstoqueMinihub,EstoqueNA,EstoqueTransitpoint,Bradesco Renegociação,Lista de presentes,Pague Lojas,Piloto Seguro,REG125
0,NO,student,2104482816,4482816,953877,ALINE ZUVERANA GOMES,ASSESSOR DE VENDAS,10.08.1987,7716689614,#Vendedor,...,,,,,,,,,,
1,NO,student,2100240095,240095,285815,FRANCISCA ZULENE DE MELO,VENDEDOR INTERNO,30.06.1978,8084588761,#Vendedor,...,,,,,,,,,,


In [1622]:
print(df_Gerencial_final['action'].value_counts())

NO        33627
CREATE       63
Name: action, dtype: int64


In [1623]:
##Criando a senha de acesso por usuario!
df_usuarios['cpf'] = df_usuarios['cpf'].astype(float)
df_usuarios['cpf'] = df_usuarios['cpf'].apply(lambda x: '%.0f' %x)
df_usuarios['cpf'] = df_usuarios['cpf'].astype(str).str.zfill(11)
df_usuarios ['CPF_base'] = [x[:4] for x in  df_usuarios['cpf']]
df_usuarios['nascimento'] = df_usuarios['nascimento'].astype(str)
df_usuarios['dia'] = df_usuarios['nascimento'].apply(lambda x: x[:2])
df_usuarios['password'] = df_usuarios['CPF_base'] + df_usuarios['dia']

In [1624]:
##Excluindo o os não encontrados
df_usuarios= df_usuarios.replace(np.nan, '0')

In [1625]:
##Criando o e-mail dos funcionarios
df_usuarios ['email_base'] = '@viaapp.io'
df_usuarios ['email'] = df_usuarios['user_id'].astype(str) + df_usuarios ['email_base']
df_usuarios ['email'] = df_usuarios['username'].astype(str) + df_usuarios ['email_base']
df_usuarios ['role']

0        student
1        student
2        student
3        student
4        student
          ...   
33685    student
33686    student
33687    student
33688    student
33689    student
Name: role, Length: 33690, dtype: object

In [1626]:
df_usuarios.shape

(33690, 40)

In [1627]:
## Juntar os times por usuarios 
df_usuarios['cod'] = df_usuarios['username'].apply(lambda x: x[:2])
df_usuarios['teams'] = '#Via'
df_usuarios['teams'] = np.where(df_usuarios['cod'] == '21',df_usuarios['teams'],'Parceiros Terceirizados')
df_usuarios['teams'] = df_usuarios['teams']

In [1628]:
#Ajustando o time de diretoria
df_usuarios['cod'] = df_usuarios['username'].apply(lambda x: x[:2])
df_usuarios['Diretoria'] = df_usuarios['diretoria'].astype(str).str.zfill(2)
df_usuarios['teams'] = np.where(df_usuarios['Diretoria'] == '0-',df_usuarios['teams'],df_usuarios['teams'] + '//'+'DIR'+ df_usuarios['Diretoria'])

In [1629]:
#trazendo os times antigos para comparativo
df_comp = df_user_id.copy()
df_comp = df_comp.rename(columns = {'Username':'username', 'role':'role_ant', 'email':'email_ant', 'name':'name_ant'})
df_comp = df_comp[['username', 'team', 'role_ant', 'email_ant', 'name_ant']]

df_usuarios = df_usuarios.merge(df_comp, on = 'username', how = 'left')
df_usuarios['team'] = df_usuarios['team'].astype(str)

In [1630]:
#print(df_usuarios['action'].value_counts())

In [1631]:
times_personalizados

array(['#Obrigatórios', 'Lista de presentes', 'REG125',
       '*Teste Piloto Missões Soluções', 'EstoqueNA', 'Pague Lojas',
       'EstoqueDropoff', 'EstoqueMinihub', 'EstoqueTransitpoint', 'Cipa',
       'Piloto Seguro', 'Bradesco Renegociação',
       'Bradesco Renegociação Gestor', 'BVBV Vendas CB', 'BVBV Vendas PT',
       'BVBV Atendimento CB', 'BVBV Atendimento PT'], dtype=object)

In [1632]:
#Ajustando demais times

TIMES = ['TIME_GALERIA', 'SISTEMA_PROVE', 'CATEGORIA', 'GALERIA', 'PONTO_ONLINE', 'BANDEIRA']
TIMES = [*TIMES, *times_personalizados] #Adicionando as colunas dos times personalizados
VAZIOS = ['0-', '-', '', '0', 'Não Elegível', 'Não Elegivel']

for time in TIMES:
    df_usuarios['teams'] = np.where(df_usuarios[time].isin(VAZIOS),df_usuarios['teams'],df_usuarios['teams'] + '//' + df_usuarios[time])
    df_usuarios['check_update'] = df_usuarios.apply(lambda x: True if x[time] in VAZIOS else x[time] in x.team , axis=1)
    df_usuarios['action'] = np.where((df_usuarios['check_update'] == False) & (df_usuarios['action'] != 'CREATE') , 'OVERWRITE',df_usuarios['action'] )

In [1633]:
df_usuarios.head(5)

,action,role,username,matricula,user_id,name,cargo,nascimento,cpf,TIME_GALERIA,...,email_base,email,cod,teams,Diretoria,team,role_ant,email_ant,name_ant,check_update
0,NO,student,2104482816,4482816,953877,ALINE ZUVERANA GOMES,ASSESSOR DE VENDAS,10.08.1987,07716689614,#Vendedor,...,@viaapp.io,2104482816@viaapp.io,21,#Via//DIR04//#Vendedor//Sistema Prove//Tecnolo...,04,Tecnologia//#CasasBahia//Ponto Online//DIR04//...,student,2104482816@viaapp.io,ALINE ZUVERANA GOMES,True
1,OVERWRITE,student,2100240095,240095,285815,FRANCISCA ZULENE DE MELO,VENDEDOR INTERNO,30.06.1978,08084588761,#Vendedor,...,@viaapp.io,2100240095@viaapp.io,21,#Via//DIR02//#Vendedor//Sistema Prove//Linha L...,02,Nova Galeria//#CasasBahia//Pague Lojas//#Via//...,student,2100240095@viaapp.io,FRANCISCA ZULENE DE MELO,True
2,NO,student,2100240095,240095,285815,FRANCISCA ZULENE DE MELO,VENDEDOR INTERNO,30.06.1978,08084588761,#Vendedor,...,@viaapp.io,2100240095@viaapp.io,21,#Via//DIR02//#Vendedor//Sistema Prove//Linha B...,02,Nova Galeria//#CasasBahia//Pague Lojas//#Via//...,student,2100240095@viaapp.io,FRANCISCA ZULENE DE MELO,True
3,NO,student,2101839616,1839616,402426,MARIA ZULEIDE DA COSTA XAVIER,CAIXA,27.04.1967,35711167391,Caixa,...,@viaapp.io,2101839616@viaapp.io,21,#Via//DIR07//Caixa//Sistema Prove//Nova Galeri...,07,Nova Galeria//#Via//DIR07//Caixa//Ponto Online...,student,2101839616@viaapp.io,MARIA ZULEIDE DA COSTA XAVIER,True
4,NO,student,2104908007,4908007,1248777,REGINA ZORRILLA,CONS ADM LOJA,14.12.1985,02276304162,CAL,...,@viaapp.io,2104908007@viaapp.io,21,#Via//DIR04//CAL//Sistema Prove//Nova Galeria/...,04,Ponto Online//CAL//Sistema Prove//Nova Galeria...,student,2104908007@viaapp.io,REGINA ZORRILLA,True


In [1634]:
#print(df_usuarios['action'].value_counts())

In [1635]:
df_usuarios.shape

(33690, 48)

In [1636]:
#Atualizando os admins
df_usuarios['role'] = np.where(df_usuarios['role_ant'] == 'admin', 'admin',df_usuarios['role'] )
df_usuarios['teams'] = np.where(df_usuarios['role_ant'] == 'admin', df_usuarios['teams'] + '//#ADM',df_usuarios['teams'] )
df_usuarios['email'] = np.where((df_usuarios['email_ant'] != df_usuarios['email'])&(df_usuarios['role_ant'] == 'admin') , df_usuarios['email_ant'],df_usuarios['email'])
df_usuarios['name'] = np.where((df_usuarios['name_ant'] != df_usuarios['name'])&(df_usuarios['role_ant'] == 'admin') , df_usuarios['name_ant'],df_usuarios['name'])

In [1637]:
df_usuarios.shape

(33690, 48)

In [1638]:
#Filtrando apenas quem tem action e ajustando o id para o Overwrite
df_usuarios['action'] = df_usuarios['action'].replace("", "NO")
df_usuarios = df_usuarios[df_usuarios['action'] != 'NO']

df_usuarios = df_usuarios.rename(columns = {'user_id': 'id'})

df_usuarios['id'] = np.where(df_usuarios['action'] == 'OVERWRITE', df_usuarios['username'], '')

In [1639]:
df_usuarios.shape

(2734, 48)

In [1640]:
##Aplicando padrão
df_usuarios['leaders'] = ""

df_usuarios['language'] = "pt-BR"

df_usuarios['regional'] = df_usuarios['regional'].astype(str)

df_usuarios['cargo'] = df_usuarios['cargo'].astype(str)

df_usuarios = df_usuarios[['action', 'id','username', 'email', 'name', 'role', 'password','teams', 'leaders','language', 'regional','cargo']]

In [1641]:
#Comparando a base para trazer quem deve entrar no time Excluir
df_user_id = df_user_id.merge(df_Oficial_fun[['username', 'User_id']], on = 'username', how = 'left')
df_user_id = df_user_id[df_user_id['User_id'].isnull()]
df_user_id = df_user_id[['username', 'name', 'email', 'role']]
df_user_id = df_user_id[df_user_id['role'] != 'admin']

df_user_id['action'] = 'OVERWRITE'
df_user_id['id'] = df_user_id['username']
df_user_id['password'] = 'exclui'
df_user_id['teams'] = 'Excluir'
df_user_id['leaders'] = ''
df_user_id['language'] = "pt-BR"
df_user_id['regional'] = '-'
df_user_id['cargo'] = '-'

df_user_id = df_user_id[['action', 'id','username', 'email', 'name', 'role', 'password','teams', 'leaders','language', 'regional','cargo']]

In [1642]:
df_user_id = df_user_id[df_user_id['role'] == 'student']

In [1643]:
df_usuarios = pd.concat([df_usuarios, df_user_id], axis = 0, ignore_index=True)

In [1644]:
df_usuarios.shape

(8794, 12)

In [1645]:
df_usuarios['username'].nunique()

8706

In [1646]:
if df_cadastro_manual.shape[0] > 0:
    df_usuarios = pd.concat([df_usuarios, df_cadastro_manual], axis = 0, ignore_index=True)
    df_usuarios = df_usuarios.replace(np.nan, '')

In [1647]:
#tirando usuários que já estão como excluir na skore

In [1648]:
df_user_skore = pd.read_csv('./input/{}/username_userid.csv'.format(MONTH), sep = ';',encoding = 'latin-1')

In [1649]:
df_excluir = pd.merge(df_usuarios, df_user_skore, on = 'username', how='left' )

In [1650]:
df_excluir['excluir_ajuste'] = np.where((df_excluir['teams'] =='Excluir') & (df_excluir['team']=='Excluir'),'duplicado','ok')

In [1651]:
df_excluir = df_excluir[['action', 'id_x', 'username', 'email_x','name_x','role_x', 'password', 'teams', 'leaders', 'language', 'regional', 'cargo', 'excluir_ajuste']]

In [1652]:
df_excluir = df_excluir.rename(columns = {'id_x':'id',
                                          'email_x': 'email',
                                          'name_x': 'name',
                                          'role_x': 'role'
                                         })

In [1653]:
df_duplicados_excluir = df_excluir[df_excluir['excluir_ajuste']=='duplicado']

In [1654]:
df_duplicados_excluir.to_excel('./output/excluir/df_usuarios_excluidos_skore.xlsx', index=False)

In [1655]:
df_usuarios = df_excluir[df_excluir['excluir_ajuste']=='ok']

In [1656]:
df_usuarios = df_usuarios.drop('excluir_ajuste', axis =1)

In [1657]:
df_usuarios = df_usuarios.sort_values(by=['action'],ascending=[True])

In [1658]:
#tirando usuários que não tiveram mudança no time

In [1659]:
df_user_skore = pd.read_csv('./input/{}/username_userid.csv'.format(MONTH), sep = ';',encoding = 'latin-1')

In [1660]:
df_usuarios_overwrite = df_usuarios.merge(df_user_skore, on = 'username', how='left')

In [1661]:
v = np.sort(df_usuarios_overwrite.teams.str.split('//', expand=True).fillna(''), axis=1)
df_sort = pd.DataFrame(v).agg('//'.join, 1).str.strip('//')
df_usuarios_overwrite['teams'] = df_sort

In [1662]:
v = np.sort(df_usuarios_overwrite.team.str.split('//', expand=True).fillna(''), axis=1)
df_sort = pd.DataFrame(v).agg('//'.join, 1).str.strip('//')
df_usuarios_overwrite['team'] = df_sort

In [1663]:
df_usuarios_overwrite['teams'] = df_usuarios_overwrite['teams'].str.replace('//BVBV Vendas CB//BVBV Vendas CB//','//BVBV Vendas CB//')
df_usuarios_overwrite['teams'] = df_usuarios_overwrite['teams'].str.replace('//BVBV Atendimento CB//BVBV Atendimento CB//','//BVBV Atendimento CB//')
df_usuarios_overwrite['teams'] = df_usuarios_overwrite['teams'].str.replace('//BVBV Vendas PT//BVBV Vendas PT//','//BVBV Vendas PT//')
df_usuarios_overwrite['teams'] = df_usuarios_overwrite['teams'].str.replace('//BVBV Atendimento PT//BVBV Atendimento PT//','//BVBV Atendimento PT//')

In [1664]:
df_usuarios_overwrite['teams'] = df_usuarios_overwrite['teams'].str.replace('CORP', 'Corp', regex=True)

In [1665]:
df_usuarios_overwrite['team_aux'] = np.where(df_usuarios_overwrite['team']==df_usuarios_overwrite['teams'],'igual',0)

In [1666]:
df_usuarios_overwrite = df_usuarios_overwrite[df_usuarios_overwrite['team_aux']=='0']

In [1667]:
df_usuarios = df_usuarios_overwrite[['action','id_x', 'username','email_x','name_x','role_x','password','teams','leaders','language','regional','cargo']]

In [1668]:
df_usuarios = df_usuarios.rename(columns={'id_x': 'id', 'email_x': 'email', 'name_x': 'name', 'role_x':'role'})

In [1669]:
###particionar arquivo

In [1670]:
# if particoes.lower() == 'sim' or particoes.lower() == 's':
#     quantidade = int(input('Qual o intervalo das particoes ? '))
#     part = df_usuarios.shape[0]/quantidade
#     dfs = np.array_split(df_usuarios, part)
#     part = int(part)
# else:
#     part = 0
    
# i = 0
# while i < part:
#     dfs[i].to_csv('./output/usuarios_prove_{}{}.csv'.format(TODAY, [i]), index = False, encoding = 'Utf-8')
#     i += 1

In [1671]:
df_usuarios.to_csv('./output/usuarios_prove_{}.csv'.format(TODAY), index = False, encoding = 'Utf-8')